### 輸出
IDF

TFIDF

keywords dictionary

keywords sets

In [1]:
import pymongo
from pymongo import MongoClient
import jieba
import math
import operator
import json
import jieba.analyse
import time
import os
from collections import Counter

version = 1 #
startT = time.time()
src = "Titlize_all" # DB 讀取來源 collection
filePath = "C:\\Users\\Java\\Desktop\\TM"
focus = "allTags" # 欲加強的目標
multi = 100  # 加強目標的權重
nowtime = "0927"#time.strftime("%m%d")
kwNum = 5

# "Kyoto_Titlize"
# "渡月橋" Togetsukyō
# "平等院鳳凰堂"  Byōdōin
# "千本鳥居" Fushimi-Inari Taisha
# "京都御所" Kyōto gosho
# "京都塔" kyoto tower
# "二條城" Nijō-jō
# "三十三間堂" Sanjūsangen-dō
# "清水寺" Kiyomizu-dera
# "金閣寺" Rokuonji

# focus list:
# TAGactivity.txt
# TAGsite.txt
# TAGrestaurant.txt
# TAGshopping.txt
# TAGwearing.txt

# user dict

# jieba.load_userdict(filePath + "\\TEST.txt")
jieba.load_userdict(filePath + "\\userDict\\dict.txt") # taiba字典
jieba.load_userdict(filePath + "\\userDict\\hotel.txt")
jieba.load_userdict(filePath + "\\userDict\\people.txt")
jieba.load_userdict(filePath + "\\userDict\\stopwords.txt") # stopwords先當作字典讓 jieba 切出來，之後再排除
jieba.load_userdict(filePath + "\\userDict\\京都市町名.txt")
jieba.load_userdict(filePath + "\\userDict\\京都駅名.txt")

jieba.load_userdict(filePath + "\\userDict\\tags\\TAGactivity.txt")
jieba.load_userdict(filePath + "\\userDict\\tags\\TAGsite.txt")
jieba.load_userdict(filePath + "\\userDict\\tags\\TAGrestaurant.txt")
jieba.load_userdict(filePath + "\\userDict\\tags\\TAGshopping.txt")
jieba.load_userdict(filePath + "\\userDict\\tags\\TAGwearing.txt")

# stop dict

# 載入focus
g = open(filePath + "\\userDict\\tags\\{}.txt".format(focus), 'r', encoding='utf-8')
focused = g.readlines()
site = []
# cnt=0
for item in focused:
    site.append(item.strip())
g.close()

# stop dict
f = open(filePath + "\\userDict\\stopwords.txt", 'r', encoding='utf-8')
stopwords = f.readlines()
ignore = []
# cnt=0
for item in stopwords:
    ignore.append(item.strip())
f.close()

# read data
conn = MongoClient('127.0.0.1', 27017)  ##連 Mongodb
db = conn.text_mining  ##創造DB  改程式碼的時候注意一下 這行的DB也要改
db.authenticate('text_mining', 'bb102', source='text_mining')  # text_mining帳號 #bb102密碼 #test123允許儲存的db
collection = db[src]  ##創造collection
articleList = []
for item in collection.find({}, {"內文": 1, "_id": 0}):  # for others
    if item != {}:
        articleList.append(item)

articles = []
for atc in articleList:
    article1 = atc.get('內文') #.split("※ 發信站")[0].lower()
    #article2 = "".join([i for i in filter(lambda ch: ch not in ignore, article1)])  # 去掉數字和特殊符號              
    articles.append(article1.replace(" ","").replace("　", "").replace(u'\u200b', ''))

urls = []
for item in collection.find({}, {"url": 1, "_id": 0}):  # for others
    if item != {}:
        urls.append(item)
        
titles = []
for item in collection.find({}, {"標題": 1, "_id": 0}):  # for others
    if item != {}:
        titles.append(item)

dbReadT = time.time()
        
# # article string -> term freq (tf)

def atc2tf(article):
    cut_line = jieba.cut(article)
    cnt = 0
    tf_dict = {}
    for word in cut_line:
        cnt = cnt + 1
        if word not in ignore: # 篩選 by 自定義的 stop words
            #print("key: " + str(word))# + "value:" + str(tf_dict.get(word)))
            if word in site:
                if word in tf_dict:
                    tf_dict[word] += 1*multi
                else:
                    tf_dict[word] = 1*multi
            else:
                if word in tf_dict:
                    tf_dict[word] += 1
                else:
                    tf_dict[word] = 1
    for word in tf_dict:
        #print("key: " + str(word) + "value:" + str(tf_dict.get(word)))
        tf_dict[word] = tf_dict.get(word)/cnt
    return tf_dict

# articles -> inverse document freq (idf)

def atcs2idf(articles):
    idf_dict = {}
    all_dict = {}
    total = 0
    for article in articles:
        total += 1
        tmp_dict = {}
        cut_line = jieba.cut(article)
        for word in cut_line:
            if word in ignore:
                continue
            tmp_dict[word] = 1
        for key in tmp_dict:
            num = all_dict.get(key, 0)  # 用key搜尋value，0是此key不存在時的回傳值
            all_dict[key] = num + 1
    for key in all_dict:
        p = math.log10(total/(all_dict[key]))
        idf_dict[key] = p
    return idf_dict

# atcs2idf + atc2tf -> atc_tfidf

def atc2tfidf(article, idf):
    tfidf_dict = {}
    tf = atc2tf(article)
    for word in tf:
        tfidf_dict[word] = tf[word] * idf[word]
    return tfidf_dict


#read idf files (a dictionary)
#f = open(filePath + "\\IDF\\{}kw-{}-weight-{}-{}IDF{}.txt".format(15, multi, focus, str(src), nowtime), 'r', encoding='utf-8')
f = open(filePath + "\\IDF\\{}-weight-{}-{}-IDF{}.txt".format( multi, focus, str(src), nowtime), 'r', encoding='utf-8')
_idf = json.loads(f.read())
f.close()

# 用 atcs2idf function 建立 idf 文件 dictionary
# _idf = atcs2idf(articles)

idfT = time.time()

# 將建立好的 idf 儲存為 txt
# if os.path.exists(filePath + "\\IDF\\") == False:
#     os.makedirs(filePath + "\\IDF\\")
# f = open(filePath + "\\IDF\\{}kw-{}-weight-{}-{}IDF{}.txt".format(kwNum, multi, focus, str(src), nowtime), 'w', encoding='utf-8')
# x = json.dumps(_idf, ensure_ascii = False)
# f.write(x)
# f.close()

idfSaveT = time.time()

## 輸出 df-idf
# if os.path.exists(filePath + "\\TFIDF\\") == False:
#     os.makedirs(filePath + "\\TFIDF\\")
# with open(filePath + "\\TFIDF\\{}kw-{}-weight-{}-{}TFIDF{}.txt".format(kwNum, multi, focus, str(src), nowtime), 'w', encoding='utf-8') as f:
#     for i in range(0, len(articles)):
#         try:
#             x = atc2tfidf(articles[i], _idf)
#             sorted_x = sorted(x.items(), key=operator.itemgetter(1), reverse = True)
#             f.write("== {} ==\n".format(i))
#             f.write(str(urls[i]['url'])+"\n")
#             for j in range(0,5):
#                 f.write(str(sorted_x[j][0])+"\n")
#         except IndexError as e:
#             print(e); pass

tfidfSaveT = time.time()
        
    
print("dbRead & filtering: {}".format(dbReadT - startT))
print("idf build         : {}".format(idfT - dbReadT))
print("idf Save          : {}".format(idfSaveT - idfT))
print("tf-idf build      : {}".format(tfidfSaveT - idfSaveT))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Java\AppData\Local\Temp\jieba.cache
Loading model cost 0.596 seconds.
Prefix dict has been built succesfully.


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Java\\Desktop\\TM\\IDF\\100-weight-allTags-Titlize_allIDF0927.txt'

In [ ]:
urls[628]

In [ ]:
" / ".join(jieba.cut(articles[1]))

In [ ]:
conn.close()

##  篩選標題

In [ ]:
# import re
# from collections import Counter

In [ ]:
## db.getCollection('平等院鳳凰堂_Titlize').find({"標題":{"$regex":"問題"}})

In [ ]:
# cato = {}
# for i in titles:
#     x = "".join(re.findall("^\[(..)\]", i['標題']))
#     if x not in cato:
#         cato[x] = 1
#     else:
#         cato[x] += 1
# cato.pop("")

In [ ]:
#sorted(cato.items(), key=operator.itemgetter(1), reverse = True)

In [ ]:
#### [問題] Question = []
# cnt = 0
# Question = []
# for i in titles:
#     x = re.findall("\[問題]", i['標題']) 
#     if x == ['[問題]']:
#         Question.append(cnt)
#     cnt += 1
# # for i in titles:
# #     print(i['標題'])

In [ ]:
# #### [徵文] Election = []
# cnt = 0
# Election = []
# for i in titles:
#     x = re.findall("\[徵文]", i['標題']) 
#     if x == ['[徵文]']:
#         Election.append(cnt)
#     cnt += 1
# # for i in titles:
# #     print(i['標題'])

## 生成 kwNum 個 keywords for each article

In [ ]:
##### 每篇keywords生成一筆dictionary
##### 內涵 url & keywords
kBuildStart = time.time()
keywords = []
for i in range(0, len(articles)):
    kwd = {}
    try:
        x = atc2tfidf(articles[i], _idf)
        sorted_x = sorted(x.items(), key=operator.itemgetter(1), reverse = True)
        for j in range(0, kwNum):
            kwd[str(sorted_x[j][0])] = str(sorted_x[j][1])
        kwd['url'] = urls[i].get('url', 0)   # url
        kwd['title'] = titles[i].get('標題', 0)    # title
        keywords.append(kwd)     
    except IndexError as e:
        kwd['url'] = urls[i].get('url', 0)     # url
        kwd['title'] = titles[i].get('標題', 0)    # title
        keywords.append(kwd)     # url
        pass
kBuildDone = time.time()
print("keywords build : {}".format(kBuildDone-kBuildStart))

In [ ]:
keywords

##  Save
keywords dictionary

In [ ]:
# # 儲存 keywords
# kSaveStart = time.time()
# f = open(filePath + "\\clustering\\keywordsSets\\{}kw-{}-weight-{}-{}keywords{}.txt".format(kwNum, multi, focus, str(src), nowtime), 'w', encoding="utf-8")
# f.write(json.dumps(keywords, ensure_ascii = False))
# f.close()
# kSaveDone = time.time()
# print("keywords Save : {}".format(kSaveDone-kSaveStart))

## Save
keywords set

In [ ]:
# # 關鍵詞塞入 set 並存成txt

# distinct_kw = set()
# for item in keywords:
#     x = item.keys()
#     for i in x:
#         distinct_kw.add(i)
# kSetBuildStart = time.time()  
# f = open(filePath + "\\clustering\\keywordsSets\\{}kw-{}-weight-{}-{}kwSet{}.txt".format(kwNum, multi, focus, str(src), nowtime), 'w', encoding="utf-8")
# for i in distinct_kw:
#     f.write(i+"\n")
# f.close()
# kSetBuildDone = time.time()
# print("keywords set build : {}".format(kSetBuildDone-kSetBuildStart))

## 生成 fp growth 所需格式

In [ ]:
# g = open("C:\\Users\\Java\\Desktop\\TM\\Association\\fpgData{}.txt".format(nowtime), 'w', encoding='utf-8')

# for i in range(0, len(keywords)):
#     line = " ".join([j for j in keywords[i]])
#     g.write(line + "\n")    
# g.close()